In [0]:
dbutils.fs.put("/schenarios/comple_json.json","""[
	{
		"id": "0001",
		"type": "donut",
		"name": "Cake",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" },
						{ "id": "1003", "type": "Blueberry" },
						{ "id": "1004", "type": "Devil's Food" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5007", "type": "Powdered Sugar" },
				{ "id": "5006", "type": "Chocolate with Sprinkles" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0002",
		"type": "donut",
		"name": "Raised",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0003",
		"type": "donut",
		"name": "Old Fashioned",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	}
]""",True)

Wrote 1433 bytes.
Out[9]: True

In [0]:
df_json = spark.read.option("multiline","true").json("/schenarios/comple_json.json")
#df_json.printSchema()

In [0]:
df_json.printSchema()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [0]:
from pyspark.sql.functions import explode,col
df_json = spark.read.option("multiline","true").json("/schenarios/complex_json.json")
df_json.printSchema()
display(df_json.select(explode("batters.batter")))

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



col
"List(1001, Regular)"
"List(1002, Chocolate)"
"List(1003, Blueberry)"
"List(1004, Devil's Food)"


In [0]:
df_final = df_json.withColumn("topping_explode",explode("topping"))\
                  .withColumn("topping_id",col("topping_explode.id"))\
                  .withColumn("topping_type",col("topping_explode.type"))\
                  .drop("topping","topping_explode")\
                  .withColumn("batter_explode",explode("batters.batter"))\
                  .withColumn("batter_id",col("batter_explode.id"))\
                  .withColumn("batter_type",col("batter_explode.type"))\
                  .drop("batters","batter_explode")
display(df_final)

id,name,ppu,type,topping_id,topping_type,batter_id,batter_type
0001,Cake,0.55,donut,5001,None,1001,Regular
0001,Cake,0.55,donut,5001,None,1002,Chocolate
0001,Cake,0.55,donut,5001,None,1003,Blueberry
0001,Cake,0.55,donut,5001,None,1004,Devil's Food
0001,Cake,0.55,donut,5002,Glazed,1001,Regular
0001,Cake,0.55,donut,5002,Glazed,1002,Chocolate
0001,Cake,0.55,donut,5002,Glazed,1003,Blueberry
0001,Cake,0.55,donut,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,donut,5005,Sugar,1001,Regular
0001,Cake,0.55,donut,5005,Sugar,1002,Chocolate


In [0]:
df_final.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- type: string (nullable = true)
 |-- topping_id: string (nullable = true)
 |-- topping_type: string (nullable = true)
 |-- batter_id: string (nullable = true)
 |-- batter_type: string (nullable = true)

